In [6]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import YoloXSmall, Yolov5Small, Yolov5SmallWithPlainBscp, Yolov5Middle

In [7]:
class_num = 1

In [8]:
# yolo = Yolov5Small(3, class_num)
yolo = YoloXSmall(num_anchors=1, in_channel=3, num_classes=class_num)
# yolo = Yolov5SmallWithPlainBscp(3, class_num)
# yolo = Yolov5Middle(3, class_num)
my_state_dict = yolo.state_dict()

In [ ]:
my_total_k = []
my_total_v = []
my_focus_layer = []
my_stage_1_conv = []
for i, (k, v) in enumerate(my_state_dict.items()):
    my_total_k.append(k)
    my_total_v.append(v)
    # print(f'{i}: {k}\t\t{v.shape}')
    if "neck" in k and "focus" in k:
        my_focus_layer.append(v)
        continue
    if "neck" in k and "stage1_conv" in k:
        my_stage_1_conv.append(v)

In [10]:
v5_state_dict = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/yolov5_small_for_voc.pth', map_location='cpu')

In [11]:
v5_state_dict = v5_state_dict['model_state_dict']

In [ ]:
for i, (k, v) in enumerate(v5_state_dict.items()):
    # print(k, v.shape)
    pass

In [ ]:
v5_total_k, v5_total_v = [], []
v5_focus_layer = []
v5_stage_1_conv = []
for i, (k, v) in enumerate(v5_state_dict.items()):
    if 'anchor' not in k:
        v5_total_k.append(k)
        v5_total_v.append(v)
        # print(f'{i}: {k}\t\t{v.shape}')
    if "model.0" in k:
        v5_focus_layer.append(v)
    if "model.1" in k:
        v5_stage_1_conv.append(v)

In [14]:
len(my_total_k), len(v5_total_k)

(414, 348)

In [15]:
for i in range(min(len(my_total_k), len(v5_total_k))):
    if my_total_v[i].shape != v5_total_v[i].shape:
        print(i)

342
343
344
345
346
347


## load prarmeters

In [ ]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:  # 只load两个网络间相同layer的参数，其他参数使用默认初始化
        continue
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data = total[i]
        # print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data = total[i]
        i += 1
        m.bias.data = total[i]
        i += 1
        m.running_mean = total[i]
        i += 1
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

In [17]:
i

342

## check

In [18]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:  # 只检查上一步load参数是否一致
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [19]:
my_output_state = yolo.state_dict()
torch.save({'model_state_dict': my_output_state}, "/home/uih/JYL/Programs/YOLO_ckpts/yolox_small_for_wheat.pth")